In [1]:
import numpy as np
import pandas as pd
from itertools import combinations
import pyEDM
import seaborn as sns
from joblib import Parallel, delayed
from numpy.random import default_rng
from scipy.spatial.distance import squareform, pdist
from sklearn.preprocessing import StandardScaler



def recurrence_matrix(x_e, quantile_per=12.5):
    """
    Generates binary recurrence matrix, where the closest ``quantile_per``% of pairwise distances are assigned a 1,
    and all other pairs assigned a 0.
    :param x_e: embedded dataframe of the required form such as that generated through ``pyEDM.Embed()``
    :param quantile_per: percentile of distances to set equal to 1
    :return: dataframe of recurrence matrix
    """
    dist_mat = squareform(pdist(x_e.values, metric="chebyshev"))
    binary_dist_mtx = (dist_mat <= np.percentile(dist_mat, quantile_per)).astype(int)
    # print("1s: ", np.count_nonzero(binary_dist_mtx) / (len(binary_dist_mtx) ** 2))
    return binary_dist_mtx

def twins_list(len_df, Ng, r_mtx, obs_per_year=12):
    """
    Creates an array of all twin points. Twin points are defined as those which are sufficiently close together in
    state space (same column in the recurrence matrix) and have the same seasonality (measurements from the same month)
    :param len_df: length of original dataframe (i.e. length of time series)
    :param Ng: length of embedded dataframe
    :param r_mtx: recurrence matrix dataframe
    :param obs_per_year: seasonality of the data
    :return: array of twins, formatted in two different ways. A_arr is the useful format for what follows
    """
    Eminus1 = len_df - Ng
    Alist = [[i] for i in range(len_df)]
    for i, j in combinations(range(Ng), 2):  # does not include repeats
        if np.array_equal(r_mtx[:, i], r_mtx[:, j]) and (j - i) % obs_per_year == 0:
            Alist[i + Eminus1].append(j + Eminus1)
            Alist[j + Eminus1].append(i + Eminus1)
    Q = np.array([len(twins) for twins in Alist])
    A_arr = (
        np.zeros((len_df, np.max(Q)), dtype=int) - 1000
    )  # -1000 here after trying it
    for i in range(len_df):
        A_arr[i, : Q[i]] = Alist[i]
    return A_arr, Q



def network_surrogates(len_df, twins_arr, Q, Ng, M=100, obs_per_year=12):
    """
    Generate all surrogates using a network method described in report. First generate points in phase space,
    and then append on initial time series
    :param len_df: length of original dataframe (i.e. length of time series)
    :param twins_arr: array of the twin points of every point (e.g. ouput of twins list
    :param Q: array of degrees of each node
    :param Ng: length of embedded dataframe/ number of nodes
    :param M: number of walkers starting on each node
    :param obs_per_year: sampling observations per year of data
    :return: arrays for which each column is a surrogate
    """
    nodes = np.arange(Ng - 2)
    Eminus1 = len_df - Ng
    start_nodes = nodes[nodes % obs_per_year == 0]
    start_nodes = start_nodes[start_nodes >= obs_per_year]
    num_nodes = len(start_nodes)
    X = np.zeros((Ng, M, num_nodes), dtype=int)
    X[0, :, :] = np.tile(start_nodes, (M, 1))

    rng = default_rng()  # generate random numbers (using numpy generator method)
    rand1 = rng.random((Ng, M, num_nodes))

    for i in range(1, Ng):
        X[i, :, :] = X[i - 1, :, :]
        QX = Q[X[i, :, :]]  # degrees of nodes where each walker is currently
        Ri = (
            rand1[i, :, :] * QX
        )  # Generate array of random numbers, each less than QX[i,j]
        Ri = Ri.astype(int)  # integer conversion
        mask = X[i, :, :] < (Ng - 1)  # relax this condition to let points jump off
        X[i, :, :][mask] = twins_arr[X[i, :, :], Ri][mask]  # update walker locations
        X[i, :, :][X[i, :, :] < (Ng - 1)] += 1
    surrogates = X[:, X[-1, :, :] < (Ng - 1)]
    # back propagate initial times to get to full length of time series
    initial_times = np.add(
        np.tile(surrogates[0, :] - Eminus1, (Eminus1, 1)),
        np.mgrid[:Eminus1, : len(surrogates[0, :])][0],
    )
    return np.concatenate((initial_times, surrogates))  # each column is a surrogate!!


def run_surrogates(df, mae_dict, sp, n_surrogates=5, obs_per_year=12, thresholds=-1):
    """
    Generates all the surrogates using the rEDM method. Note the ability to insert a list of
    thresholds to try. Default is the rEDM implementation list of thresholds to try.
    :param df: input dataframe
    :param mae_dict: dictionary of lists; generate with :meth:`src.processing.embedding_dimension.mae_dictionary`
    :param sp: species for which to generate surrogates
    :param n_surrogates: number of surrogates to generate
    :param obs_per_year: period of seasonality of data
    :param thresholds: list of thresholds to try. Numbers between 5 and 20 recommended.
    :return: embedded dataframe, surrogates, optimal embedding dimension
    """
    surrogate_slice = None
    optE = 0
    len_df = len(df)
    for count in range(25):
        optE = mae_dict[sp][count][0]
        x_e = pyEDM.Embed(dataFrame=df, E=optE, columns=sp)
        Ng = len(x_e)
        if thresholds == -1:
            thresholds = [
                12.5,
                12,
                11,
                10,
                9,
                8,
                7,
                6,
                5,
                15,
                16,
                17,
                18,
                19,
                20,
                4,
            ]
        twins_arr = None
        Q = None
        for thres in thresholds:
            r_mtx = recurrence_matrix(x_e, thres)
            twins_arr, Q = twins_list(
                len_df, Ng=Ng, r_mtx=r_mtx, obs_per_year=obs_per_year
            )
            # if there are at least 10 twins
            if np.sum(Q) - Ng > 10:
                break
        surrogate_slice = network_surrogates(
            len_df, twins_arr, Q, Ng, M=100 * n_surrogates, obs_per_year=obs_per_year
        )
        if len(surrogate_slice[0, :]) >= n_surrogates:
            print("E = {}".format(optE))
            print("Threshold = {}".format(thres))
            break
    rng = default_rng()
    return (
        x_e,
        rng.choice(a=surrogate_slice, size=n_surrogates, replace=False, axis=1),
        optE,
    )


def generate_all_surrogates(df, mae_dict, n_surrogates=5, obs_per_year=12):
    """
    Generates surrogates for all species of a given dataframe, using the :meth:`src.PLTS.networkccm3.run_surrogates`
    method.
    :param df: input dataframe
    :param mae_dict: dictionary of lists; generate with :meth:`src.processing.embedding_dimension.mae_dictionary`
    :param n_surrogates: number of surrogates to use with phase-lock twin surrogate method
    :param obs_per_year: periodicity of time series data (12 for Guadalquivir data, 24 for Maizuru data)
    :return: length of embedded dataframe, array of surrogate time series, array of optimal embedding dimensions
    """
    species_list = list(df.columns[1:])
    len_df = len(df)
    surr_array = np.zeros((len(species_list), len_df, n_surrogates))
    Es = np.zeros(len(species_list))
    for i, sp in enumerate(species_list):
        x_e, surr_array[i], Es[i] = run_surrogates(
            df, mae_dict, sp, n_surrogates=n_surrogates, obs_per_year=obs_per_year
        )
    return x_e, surr_array, Es.astype(int)



def mae_dictionary(df, max_dim=24):
    """
    Creates dictionary with species as keys and each corresponding index a list of embedding dimensions from 1 to
    max_dim in order of optimality according to mean absolute error. See section 3.2 of report.
    :param df: input dataframe of usual form (e.g. that given by :meth:`src.abundance_tools.initialise_dataset`)
    :param max_dim: maximum embedding dimension to include in dictionary. Default is 24, copying Ushio paper
    :return: dictionary of optimal embedding dimension information
    """
    mae_dict = {}
    for _, sp in enumerate(df.columns[1:]):
        MAEs = [[] for _ in range(2, max_dim + 1)]
        for E in range(2, max_dim + 1):
            library_string = "1 {}".format(len(df) - E)
            preds = pyEDM.Simplex(
                dataFrame=df,
                columns=sp,
                target=sp,
                E=E,
                Tp=1,
                lib=library_string,
                pred=library_string,
                exclusionRadius=1,
            )
            MAEs[E - 2] = [
                E,
                np.nanmean(
                    np.abs((preds["Predictions"] - preds["Observations"]).values)
                ),
            ]
        mae_dict[sp] = MAEs
    for sp in mae_dict:
        mae_dict[sp].sort(key=takeSecond)
    return mae_dict

def takeSecond(elem):
    """
    Returns second element of a list. Auxiliary function for :meth:`src.processing.embedding_dimension.mae_dictionary`
    :param elem: list
    :return: second element
    """
    return elem[1]


def twin_surrogate_esn(X, Y, Nsurr, period):
    df = pd.DataFrame()
    df['X'] = X
    df['Y'] = Y
    df.insert(loc=0, column="Time", value=range(len(df)))
    
    df_mae_dict = mae_dictionary(df, max_dim=24)
    _, surr_array, _ = generate_all_surrogates(df, df_mae_dict, n_surrogates = Nsurr, obs_per_year = period)
    
    return surr_array[0,:,:].T, surr_array[1,:,:].T

def DeepESN_real(X, Y, n_data = 50, period = 1, verbose = False):
    
    # Get length of time series
    N = X.shape[0]
    
    # compute surrogates
    Xsurr, Ysurr = twin_surrogate_esn(X, Y, Nsurr = n_data, period = period)
    
    
    # run the deepESN of the surrogates
    xmapy_surr, ymapx_surr = compute_confidence(Xsurr, Ysurr, verbose)

    # run deepESN for actual data
    xmapy, ymapx = compute_lags(X, Y, n_data, verbose)

    return xmapy_surr, ymapx_surr, xmapy, ymapx

In [2]:
maizuru = pd.read_csv('Maizuru_dominant_sp.csv')
maizuru

,date_tag,surf.t,bot.t,Aurelia.sp,Engraulis.japonicus,Plotosus.lineatus,Sebastes.inermis,Trachurus.japonicus,Girella.punctata,Pseudolabrus.sieboldi,...,Halichoeres.tenuispinnis,Chaenogobius.gulosus,Pterogobius.zonoleucus,Tridentiger.trigonocephalus,Siganus.fuscescens,Sphyraena.pinguis,Rudarius.ercodes,Y,M,D
0,2002-6-early,22.0,19.0,79.200000,0,1,463,0,60,18,...,23,470.0,0.0,7.0,0,0,7.0,2002,6,6
1,2002-6-late,21.0,21.0,157.200000,0,0,19,650,42,22,...,23,0.0,0.0,0.0,0,0,7.0,2002,6,26
2,2002-7-early,25.0,22.0,21.333333,0,300,47,1150,8,7,...,32,0.0,50.0,2.0,0,0,1.0,2002,7,12
3,2002-7-late,30.0,24.0,0.937500,0,302,46,958,34,19,...,22,80.0,6.0,38.0,0,0,1.0,2002,7,29
4,2002-8-early,28.0,25.0,19.333333,0,540,43,308,15,14,...,16,0.0,50.0,43.0,0,70,7.0,2002,8,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,2014-2-early,5.4,8.5,0.000000,150,0,1,0,0,0,...,0,0.0,0.0,1.0,0,0,1.0,2014,2,12
281,2014-2-late,5.5,10.4,0.000000,1,0,0,0,0,0,...,0,0.0,0.0,2.0,0,0,0.0,2014,2,23
282,2014-3-early,8.4,10.5,0.000000,0,0,0,0,0,0,...,0,0.0,0.0,6.0,0,0,0.0,2014,3,3
283,2014-3-late,11.5,10.4,0.250000,0,0,0,0,0,0,...,0,0.0,0.0,11.0,0,0,0.0,2014,3,23


In [3]:
maizuru.drop(['date_tag', 'surf.t', 'bot.t', 'Y', 'M', 'D'], inplace = True, axis = 1)
maizuru

,Aurelia.sp,Engraulis.japonicus,Plotosus.lineatus,Sebastes.inermis,Trachurus.japonicus,Girella.punctata,Pseudolabrus.sieboldi,Halichoeres.poecilopterus,Halichoeres.tenuispinnis,Chaenogobius.gulosus,Pterogobius.zonoleucus,Tridentiger.trigonocephalus,Siganus.fuscescens,Sphyraena.pinguis,Rudarius.ercodes
0,79.200000,0,1,463,0,60,18,31,23,470.0,0.0,7.0,0,0,7.0
1,157.200000,0,0,19,650,42,22,10,23,0.0,0.0,0.0,0,0,7.0
2,21.333333,0,300,47,1150,8,7,7,32,0.0,50.0,2.0,0,0,1.0
3,0.937500,0,302,46,958,34,19,30,22,80.0,6.0,38.0,0,0,1.0
4,19.333333,0,540,43,308,15,14,10,16,0.0,50.0,43.0,0,70,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,0.000000,150,0,1,0,0,0,0,0,0.0,0.0,1.0,0,0,1.0
281,0.000000,1,0,0,0,0,0,0,0,0.0,0.0,2.0,0,0,0.0
282,0.000000,0,0,0,0,0,0,0,0,0.0,0.0,6.0,0,0,0.0
283,0.250000,0,0,0,0,0,0,0,0,0.0,0.0,11.0,0,0,0.0


In [4]:
maizuru.insert(loc=0, column="Time", value=range(len(maizuru)))

In [5]:
maizuru

,Time,Aurelia.sp,Engraulis.japonicus,Plotosus.lineatus,Sebastes.inermis,Trachurus.japonicus,Girella.punctata,Pseudolabrus.sieboldi,Halichoeres.poecilopterus,Halichoeres.tenuispinnis,Chaenogobius.gulosus,Pterogobius.zonoleucus,Tridentiger.trigonocephalus,Siganus.fuscescens,Sphyraena.pinguis,Rudarius.ercodes
0,0,79.200000,0,1,463,0,60,18,31,23,470.0,0.0,7.0,0,0,7.0
1,1,157.200000,0,0,19,650,42,22,10,23,0.0,0.0,0.0,0,0,7.0
2,2,21.333333,0,300,47,1150,8,7,7,32,0.0,50.0,2.0,0,0,1.0
3,3,0.937500,0,302,46,958,34,19,30,22,80.0,6.0,38.0,0,0,1.0
4,4,19.333333,0,540,43,308,15,14,10,16,0.0,50.0,43.0,0,70,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,280,0.000000,150,0,1,0,0,0,0,0,0.0,0.0,1.0,0,0,1.0
281,281,0.000000,1,0,0,0,0,0,0,0,0.0,0.0,2.0,0,0,0.0
282,282,0.000000,0,0,0,0,0,0,0,0,0.0,0.0,6.0,0,0,0.0
283,283,0.250000,0,0,0,0,0,0,0,0,0.0,0.0,11.0,0,0,0.0


In [6]:
mae_dict = mae_dictionary(maizuru, max_dim=24)

In [7]:
_, surrogates, _ = generate_all_surrogates(maizuru, mae_dict = mae_dict, n_surrogates=100, obs_per_year=24)

E = 11
Threshold = 12.5
E = 24
Threshold = 12.5
E = 7
Threshold = 12.5
E = 24
Threshold = 18
E = 12
Threshold = 12.5
E = 24
Threshold = 6
E = 9
Threshold = 4
E = 12
Threshold = 4
E = 5
Threshold = 12.5
E = 6
Threshold = 12.5
E = 8
Threshold = 12.5
E = 6
Threshold = 4
E = 9
Threshold = 12.5
E = 6
Threshold = 12.5
E = 2
Threshold = 12.5


In [8]:
surrogates.shape

(15, 285, 100)

In [10]:
names = maizuru.columns.values.tolist()
names.remove('Time')
names

['Aurelia.sp',
 'Engraulis.japonicus',
 'Plotosus.lineatus',
 'Sebastes.inermis',
 'Trachurus.japonicus',
 'Girella.punctata',
 'Pseudolabrus.sieboldi',
 'Halichoeres.poecilopterus',
 'Halichoeres.tenuispinnis',
 'Chaenogobius.gulosus',
 'Pterogobius.zonoleucus',
 'Tridentiger.trigonocephalus',
 'Siganus.fuscescens',
 'Sphyraena.pinguis',
 'Rudarius.ercodes']

In [11]:
len(names)

15

In [12]:
for i, name in enumerate(names):
    np.save(name + '_surr', surrogates[i,:,:].T)

In [23]:
# np.save('Rudarius.ercodes' + '_surr', surrogates[14,:,:].T)